In [1]:
import numpy as np
import pandas as pd
import json
import collections
from collections import defaultdict 
from functools import partial
import networkx as nx
import matplotlib.pyplot as plt
import seaborn as sns
from tqdm import tqdm


In [2]:
# Good read: https://iq-inc.com/importerror-attempted-relative-import/

import sys
import importlib
from mimicnet import concept

importlib.reload(sys.modules['mimicnet.concept'])

<module 'mimicnet.concept' from '/home/asem/GP/MIMIC-SNONET/mimicnet/concept.py'>

In [3]:
# multi_visit_mimic_dir = '/home/am8520/GP/ehr-data/mimic3-multi-visit'
multi_visit_mimic_dir = '/home/asem/GP/ehr-data/mimic3-multi-visit'
transformed_mimic_dir = '/home/asem/GP/ehr-data/mimic3-transforms'
mimic_dir = '/home/asem/GP/ehr-data/mimic3-v1.4/physionet.org/files/mimiciii/1.4'

In [4]:
D_LABITEMS = pd.read_csv(f'{mimic_dir}/D_LABITEMS.csv.gz')
D_ITEMS = pd.read_csv(f'{mimic_dir}/D_ITEMS.csv.gz')
D_TEST = pd.concat([D_LABITEMS, D_ITEMS], join='inner')
test_label_dict = dict(zip(D_TEST.ITEMID, D_TEST.LABEL))
test_cat_dict = dict(zip(D_TEST.ITEMID, D_TEST.CATEGORY))

In [5]:
static_df = pd.read_csv(f'{transformed_mimic_dir}/static_df.csv.gz')
adm_df = pd.read_csv(f'{transformed_mimic_dir}/adm_df.csv.gz')
diag_df = pd.read_csv(f'{transformed_mimic_dir}/diag_df.csv.gz')
proc_df = pd.read_csv(f'{transformed_mimic_dir}/proc_df.csv.gz')
test_df = pd.read_csv(f'{transformed_mimic_dir}/test_df.csv.gz')


In [7]:
static_df.ETHNIC_GROUP.nunique()

7

In [8]:
diag_df

,SUBJECT_ID,HADM_ID,ICD9_CODE
0,112,174105,53100
1,112,174105,41071
2,112,174105,2859
3,112,174105,41401
4,112,174105,725
...,...,...,...
122612,97488,161999,0414
122613,97488,161999,30391
122614,97488,161999,E8798
122615,97488,161999,78791


In [9]:
test_df

,SUBJECT_ID,ITEMID,DATE,VALUE
0,17,50852,2134-12-22,-1.205210
1,17,50861,2134-12-22,-0.813846
2,17,50862,2134-12-22,1.632823
3,17,50863,2134-12-22,-1.450474
4,17,50867,2134-12-22,-0.611788
...,...,...,...,...
4339087,99982,227456,2157-02-22,1.820554
4339088,99982,227457,2157-02-22,-0.723138
4339089,99982,227465,2157-02-22,2.180053
4339090,99982,227466,2157-02-22,-0.036611
